scrape categories and all books for each category and then insert into mongo db

In [23]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import pymongo
import requests
import pandas as pd
from sqlalchemy import create_engine
import time

In [64]:
browser = Browser('chrome')
url = 'http://quotes.toscrape.com/'

In [66]:
browser.visit(url)

quote_list = []
each_tag_list = []
author_list = []
all_data = []

born_list = []
author_desc_list = []

quote_id = 1
for x in range(0,10):
    
    html = browser.html
    soup = bs(html, 'html.parser')
    results = soup.find_all('div', class_ = 'quote')
    for result in results:
        quotes = result.span.text
        quote_list.append(quotes)

        each_tag = result.find('meta')['content']
        each_tag = each_tag.split(',')
        each_tag_list.append(each_tag)

        authors = result.find('small').text
        author_list.append(authors)
        
        href = result.find('a')['href']
        author_url = url + href 
        response = requests.get(author_url)
        soup2 = bs(response.text, 'html.parser')
        
        born_date = soup2.find('span', class_ = 'author-born-date').text
        born_location = soup2.find('span', class_ = 'author-born-location').text
        born = born_date + ' ' + born_location
        born_list.append(born)
        description = soup2.find('div', class_ = 'author-description').text.strip()
        author_desc_list.append(description)
        
        data = {'_id': quote_id,
                'quote': quotes,
                'tags': each_tag,
                'author': authors,
                'born': born,
                'author_description': description
               }
        all_data.append(data)
        
        quote_id += 1 

    try:
        browser.links.find_by_partial_text('Next').click()
    except:
        print('Scraping complete')

all_data

[{'_id': 1,
  'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
  'author': 'Albert Einstein',
  'born': 'March 14, 1879 in Ulm, Germany',
  'author_description': 'In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world. After the rise of the Nazi party, Einstein made Princeton his permanent home, becoming a U.S. citizen in 1940. Einstein, a pacifist during World War I, stayed a firm proponent of social justice and responsibility. He chaired the Emergency Committee of Atomic Scientists, which organized to alert the public to the dangers of atomic warfare.At a symposium, he advised: "In their strugg

In [67]:
len(all_data)

100

In [75]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.quotes
collection = db.quotes
collection.insert_many(all_data)

In [69]:
browser.quit()

Extract from mongo into postgres

In [76]:
def get_quotes_from_mongo():
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client.quotes
    collection = db.quotes
    return collection

In [80]:
def normalize_quotes_data(docs):
    quotes_table = []
    authors = []
    authors_table = []
    tags_table = []
    for doc in docs.find({}):
#         print(f"normalizing the quote : [{doc['_id']}]")
        quote = {}
        quote['id'] = doc['_id']
        quote['text'] = doc['quote']
        quote['author_name'] = doc['author']
        quotes_table.append(quote)

        author = {}
        author['name'] = doc['author']
        author['born'] =  doc['born']
        author['description'] = doc['author_description']
        if (author['name'] not in authors):
            authors_table.append(author)
            authors.append(author['name'])

        for tag in doc['tags']:
            tags_table.append({'quote_id':doc['_id'] , 'tag' : tag})    
            
    return (quotes_table , authors_table ,tags_table)

In [81]:
docs = get_quotes_from_mongo()
print(f' found {docs.count_documents({})} documents')

 found 100 documents


In [82]:
(quotes , authors ,tags) = normalize_quotes_data(docs)
quotes_df = pd.DataFrame(quotes )
author_df = pd.DataFrame( authors )
tags_df = pd.DataFrame(tags)

In [101]:
author_df

,name,born,description
0,Albert Einstein,"March 14, 1879 in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
1,J.K. Rowling,"July 31, 1965 in Yate, South Gloucestershire, ...",See also: Robert GalbraithAlthough she writes ...
2,Jane Austen,"December 16, 1775 in Steventon Rectory, Hampsh...",Jane Austen was an English novelist whose work...
3,Marilyn Monroe,"June 01, 1926 in The United States",Marilyn Monroe (born Norma Jeane Mortenson; Ju...
4,André Gide,"November 22, 1869 in Paris, France",André Paul Guillaume Gide was a French author ...
5,Thomas A. Edison,"February 11, 1847 in Milan, Ohio, The United S...","Thomas Alva Edison was an American inventor, s..."
6,Eleanor Roosevelt,"October 11, 1884 in The United States",Anna Eleanor Roosevelt was an American politic...
7,Steve Martin,"August 14, 1945 in Waco, Texas, The United States","Stephen Glenn ""Steve"" Martin is an American ac..."
8,Bob Marley,"February 06, 1945 in Nine Mile, Saint Ann, Jam...","Robert ""Bob"" Nesta Marley OM was a Jamaican si..."
9,Dr. Seuss,"March 02, 1904 in Springfield, MA, The United ...",Theodor Seuss Geisel was born 2 March 1904 in ...


In [92]:
user_name = 'postgres'
password = 'letterpen'
connection_string = f"{user_name}:{password}@localhost:5432/quotes_db"
engine = create_engine(f'postgresql://{connection_string}')

In [118]:
quotes_script = '''
    create table quotes(     id INTEGER PRIMARY KEY,    
    author_name varchar(32),    
    text varchar(1500))
'''

tags_script = '''
    create table tags(    quote_id INTEGER,    
    tag varchar(32))
'''

author_script = '''
    create table author( name varchar(32) PRIMARY KEY,    
    born varchar(100),    
    description varchar(10000))
'''

In [119]:
tables = {'quotes' : quotes_script.strip(), 
          'tags' : tags_script.strip(),
          'author' : author_script.strip()
         }

In [120]:
for table in tables.keys():
    print(f'dropping the table {table} if it already exists...')
    engine.execute(f'drop table IF EXISTS {table}')

dropping the table quotes if it already exists...
dropping the table tags if it already exists...
dropping the table author if it already exists...


In [121]:
for table , script in tables.items():
    print(f'creating the table {table}...')
    engine.execute(f'{script}')

creating the table quotes...
creating the table tags...
creating the table author...


In [122]:
engine.table_names()

['author', 'quotes', 'tags']

In [123]:
quotes_df.to_sql(name='quotes', con=engine, if_exists='append', index=False)

In [124]:
tags_df.to_sql(name='tags', con=engine, if_exists='append', index=False)

In [125]:
author_df.to_sql(name='author', con=engine, if_exists='append', index=False)